# 1. BERT

In [ ]:
%%capture
!pip install datasets transformers tqdm
!pip install torch
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
import pandas as pd
import numpy as np
import ast
from datasets import Dataset
from transformers import pipeline, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/tar_projekt/csvs/dataset_train_allLevels.csv')
train_df = train_df.loc[train_df['level'] > 1]

texts = train_df['text']
labels = train_df['change']

In [ ]:
labels = [ast.literal_eval(string) for string in labels]

In [ ]:
train_df

,id,text-id,text,num-authors,level,change,text-length
4200,medium-1,1,Nevada has some very generous laws for squatte...,3,2,"[0, 0, 1, 1]",1200
4201,medium-2,2,"Sorry, we have removed your question. While it...",2,2,"[1, 1, 0, 1, 0, 1, 1]",2271
4202,medium-3,3,The issue Erdogan has with Sweden joining is n...,3,2,"[0, 1, 1]",1137
4203,medium-4,4,Capitalism is what drives women's rights. A st...,3,2,"[1, 1]",1083
4204,medium-5,5,Just continue to explain that you personally c...,2,2,"[1, 1, 1]",1176
...,...,...,...,...,...,...,...
12595,hard-4196,4196,"They can ask, but they can't make it a require...",2,3,"[0, 1, 0, 0]",1148
12596,hard-4197,4197,"In the years preceding the Iranian Revolution,...",2,3,"[0, 0, 1]",2105
12597,hard-4198,4198,Does the order from the judge say the tenant d...,4,3,"[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]",3920
12598,hard-4199,4199,The simple answer is that Western education sy...,2,3,"[0, 1, 1, 0, 0, 0, 0]",2608


In [ ]:
train_data = []

for text, label in zip(texts, labels):
    paragraphs = text.split(' ||| ')
    for (p1, p2), y in zip(zip(paragraphs, paragraphs[1: ]), label):
        p1 = p1.strip()
        p2 = p2.strip()

        train_data.append({'text': p1 + '\n' + p2, 'label': y})

In [ ]:
train_data = Dataset.from_pandas(pd.DataFrame(train_data))
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 47329
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_data.map(tokenize_function)
tokenized_train.set_format("torch")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/47329 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    do_eval = False
    #per_device_eval_batch_size=16,
    #logging_dir="./logs",
    #logging_steps=10,
    #load_best_model_at_end=True,
    #metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.678500
1000,0.658200
1500,0.647000
2000,0.630900
2500,0.621500
3000,0.604300
3500,0.610900
4000,0.633100
4500,0.601400
5000,0.608900


TrainOutput(global_step=17751, training_loss=0.5881874583471419, metrics={'train_runtime': 6132.1222, 'train_samples_per_second': 23.155, 'train_steps_per_second': 2.895, 'total_flos': 3.735834941740032e+16, 'train_loss': 0.5881874583471419, 'epoch': 3.0})

In [ ]:
# Copy the results to Drive
import locale
print(locale.getpreferredencoding())
locale.getpreferredencoding = lambda: "UTF-8"


!scp -r /content/results '/content/drive/MyDrive/tar_projekt'

UTF-8


In [ ]:
model.save_pretrained("./results_model")
tokenizer.save_pretrained("./results_model")

('./results_model/tokenizer_config.json',
 './results_model/special_tokens_map.json',
 './results_model/vocab.txt',
 './results_model/added_tokens.json',
 './results_model/tokenizer.json')

In [ ]:
!scp -r /content/results_model '/content/drive/MyDrive/tar_projekt'

In [ ]:
#model = AutoModelForSequenceClassification.from_pretrained("./results")
#tokenizer = AutoTokenizer.from_pretrained("./results")